In [1]:
import sys
sys.path.append('../../build')
import cv2 as cv
import numpy as np
import libry as ry
import time
print(cv.__version__)

** INFO:/home/tim/git/robotics-course_tim/rai/rai/Core/util.cpp:initCmdLine:602(1) ** cmd line arguments: 'rai-pybind -python'
** INFO:/home/tim/git/robotics-course_tim/rai/rai/Core/util.cpp:initCmdLine:606(1) ** run path: '/home/tim/git/robotics-course_tim/course3-Simulation/04-opencv'
**ry-c++-log** /home/tim/git/robotics-course_tim/rai/rai/ry/ry.cpp:init_LogToPythonConsole:34(0) initializing ry log callback** INFO:/home/tim/git/robotics-course_tim/rai/rai/Core/graph.cpp:initParameters:1365(1) ** parsed parameters:
{python}



4.5.3
** INFO:/home/tim/git/robotics-course_tim/rai/rai/ry/ry.cpp:init_LogToPythonConsole:34(0) initializing ry log callback


In [2]:
#-- Add REAL WORLD configuration and camera
RealWorld = ry.Config()
RealWorld.addFile("../../scenarios/ex03_redball.g")
S = RealWorld.simulation(ry.SimulatorEngine.bullet, True)
S.addSensor("camera")

C = ry.Config()
C.addFile('../../scenarios/pandasTable.g')
D = C.view()
cameraFrame = C.frame("camera")

In [3]:
#the focal length
f = 0.895
f = f * 360.
#the relative pose of the camera
# pcl.setRelativePose('d(-90 0 0 1) t(-.08 .205 .115) d(26 1 0 0) d(-1 0 1 0) d(6 0 0 1) ')
fxfypxpy = [f, f, 320., 180.]

In [4]:
obj = C.addFrame("object")
obj.setPosition([.3,0.3,.71])
#obj.setQuaternion([1,0,1,0])
obj.setShape(ry.ST.sphere, [.1])
obj.setColor([1,1,0])
obj.setContact(1)

In [5]:
# we want to optimize a single step (1 phase, 1 step/phase, duration=1, k_order=1)
komo = C.komo_path(1.,30, 5., True)
komo.addObjective([1.], ry.FS.positionDiff, ["R_gripper", "object"], ry.OT.sos, [1e2])
komo.addObjective([.5,1.], ry.FS.position, ["R_gripper"], ry.OT.eq, [1e2],[-.0,-.0,-.01], order=2)
komo.addObjective([], ry.FS.distance, ["R_gripper", "object"], ry.OT.ineq, [1e2],[-.0])

print(komo.getT(),komo.getPathFrames().shape)

komo.optimize()
#komo.getReport()
#V = komo.view_play(False, 1)

30 (30, 98, 7)
** KOMO::run solver:sparse collisions:1 x-dim:420 T:30 k:2 phases:1 stepsPerPhase:30 tau:0.166667  #timeSlices:32 #totalDOFs:420 #frames:3136
** optimization time:0.345795 (kin:0.01105 coll:0.227261 feat:0.052378 newton: 0.02695) setJointStateCount:38
   sos:100.052 ineq:0.000154054 eq:0.00323932


In [6]:

for frame,tau in zip(komo.getPathFrames(),komo.getPathTau()):
    time.sleep(tau)
    
    C.setFrameState(frame)

    q= C.getJointState()

    #send position to the simulation
    S.step(q, tau, ry.ControlMode.position)

In [7]:
def track_background(depth,rgb,background=None):
    if background is None:
        background = depth
    
    diff = background - depth

    mask = cv.inRange(diff, 0.1,100)

    _,thres = cv.threshold(diff, 0.1, 255, cv.THRESH_BINARY)

    if len(diff)>0: cv.imshow('OPENCV - mask', thres)

    filtered_mask = np.zeros(mask.shape, np.uint8)
    
    # find contours
    contours, _ = cv.findContours(mask, cv.RETR_LIST, cv.CHAIN_APPROX_SIMPLE)
    if len(contours) > 0:
        # find largest contour
        largest, idx = 0., None
        for i, c in enumerate(contours):
            # remove noise
            if c.shape[0] < 10: continue
            if cv.contourArea(c) > largest:
                largest = cv.contourArea(c)
                idx = i
        cv.drawContours(rgb, contours[idx], -1, (255, 255, 255), -1)
    if len(rgb)>0: cv.imshow('OPENCV - rgb', cv.cvtColor(rgb, cv.COLOR_BGR2RGB))
       

    # mask = cv.inRange(diff,
    return background


In [8]:
points = []
tau = .01
background = None
for t in range(400):
    time.sleep(0.03)

    #grab sensor readings from the simulation
    q = S.get_q()
    if t%10 == 0:
        [rgb, depth] = S.getImageAndDepth()  #we don't need images with 100Hz, rendering is slow
        points = S.depthData2pointCloud(depth, fxfypxpy)
        cameraFrame.setPointCloud(points, rgb)

        background = track_background(depth,rgb,background)

        if cv.waitKey(1) & 0xFF == ord('q'):
            break
        
    S.step([], tau, ry.ControlMode.none)

TypeError: list indices must be integers or slices, not NoneType

In [ ]:
#import os
#os._exit(0)